In [116]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime

import requests
import json
from bs4 import BeautifulSoup

from webdriver_manager.chrome import ChromeDriverManager

In [40]:
base = 'https://electionreturns.pa.gov'
ref = '/General/CountyBreakDownResults?officeId=1&districtId=1&ElectionID=83&ElectionType=G&IsActive=0'

In [41]:
response = requests.get(base+ref)
doc = BeautifulSoup(response.text)
doc.prettify()

'<!DOCTYPE html>\n<html lang="en" ng-app="appModule">\n <head>\n  <title>\n   Pennsylvania Elections - County Breakdown Results\n  </title>\n  <meta content="IE=edge,chrome=1" http-equiv="X-UA-Compatible"/>\n  <link href="/Content/bootstrap.css?v=1.2" rel="stylesheet" type="text/css"/>\n  <link href="/Content/bootstrap-glyphicons.css?v=1.2" rel="stylesheet" type="text/css"/>\n  <link href="/Content/core.css?v=1.2" rel="stylesheet" type="text/css"/>\n  <link href="/Content/font-awesome.css?v=1.2" rel="stylesheet" type="text/css"/>\n  <link href="/Content/angular-busy.css?v=1.2" rel="stylesheet" type="text/css"/>\n  <link href="/Content/animate.css?v=1.2" rel="stylesheet" type="text/css"/>\n  <link href="/Content/custom.css?v=1.2" rel="stylesheet" type="text/css"/>\n  <script src="/Scripts/jquery-1.12.0.min.js?v=1.2" type="text/javascript">\n  </script>\n  <script src="/Scripts/bootstrap.min.js?v=1.2" type="text/javascript">\n  </script>\n  <style>\n   [ng\\:cloak], [ng-cloak], [data-ng-

# Try Again in Selenium-- The Page is Dynamic

In [62]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

import time
import os

In [48]:
!brew install --cask chromedriver

Updating Homebrew...
==> Auto-updated Homebrew!
Updated 2 taps (homebrew/core and homebrew/cask).
==> New Formulae
all-repos                  git-svn                    mmtabbarview
aws-sso-util               gitlab-ci-local            ncnn
ca-certificates            go-critic                  neovim-qt
cassandra@3                gomodifytags               node@16
charmcraft                 goproxy                    nsh
clang-format@11            gotests                    openssl@3
clickhouse-odbc            iproute2                   postgresql@13
cloudiscovery              iputils                    python-tk@3.10
cmake-docs                 jellyfish                  python@3.10
colima                     k2tf                       qwt-qt5
copier                     lastz                      rbw
cpufetch                   ld-find-code-refs          sextractor
cyral-gimme-db-token       libnghttp2                 spdx-sbom-generator
datafusion                 libsigrok             

In [55]:
!pip install webdriver-manager

You should consider upgrading via the '/Users/shuklaa/.pyenv/versions/3.9.7/bin/python3.9 -m pip install --upgrade pip' command.


In [57]:
from webdriver_manager.chrome import ChromeDriverManager

driver = webdriver.Chrome(ChromeDriverManager().install())

[WDM] - 

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 93.0.4577
[WDM] - Get LATEST driver version for 93.0.4577
[WDM] - There is no [mac64] chromedriver for browser 93.0.4577 in cache
[WDM] - Get LATEST driver version for 93.0.4577
[WDM] - Trying to download new driver from https://chromedriver.storage.googleapis.com/93.0.4577.63/chromedriver_mac64.zip
[WDM] - Driver has been saved in cache [/Users/shuklaa/.wdm/drivers/chromedriver/mac64/93.0.4577.63]
/var/folders/ml/n5kh4d890w7_5p7h02wk78vhkfmp_0/T/ipykernel_35951/1781563638.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


In [139]:
browser = webdriver.Chrome()

browser.get(base+ref)
time.sleep(3)
html = browser.page_source
# get timestamp
ts = datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
doc = BeautifulSoup(html)

# write html to the file with the time
# define path
file_path = 'HTMLs/page_'+ts+'.html'
html = doc.prettify()
# open and save to PDFs folder with the filename we saved earlier
with open(file_path, 'w') as f:
    f.write(html)

In [109]:
# find just the table we want
table = doc.find('div',class_='container').find('div',{'ng-controller':'CountyBreakDownGeneralController'})\
    .find('div',class_='panel-body').find('div',class_='col-xs-9').find('div',class_='panel-body')\
    .find_all('div',{'ng-repeat':'districts in filterRecords(levels)'})

data = []

for elem in table:
    # get county value from header
    county = elem.find('h5').text.strip()
    # each of the 'item in district' divs represents a candidate
    cands = elem.find_all('div',{'ng-repeat':'item in districts'})
    for cand in cands:
        # one row of the final df for each candidate for each county
        row = {}
        row["County"] = county
        # each candidate contains two 'rows' of data
        r = cand.find_all('div',class_='row')
        # the first 'row' [0] contains candidate name, party, vote share, and raw votes
        details = r[0].find(class_='col-sm-4').find_all('span')
        row["Candidate"] = details[0].text.strip()
        row["Party"] = details[1].text.strip().replace("(","").replace(")","")
        row["Share"] = r[0].find(class_='col-sm-5').text.strip()
        row["Votes"] = r[0].find(class_='col-sm-3').text.split(":")[-1].strip()
        # the second 'row' [1] contains vote breakdown by method
        methods = r[1].find_all('div',class_='col-sm-4')
        row["Election Day"] = methods[0].text.split(":")[-1].strip()
        row["Mail"] = methods[1].text.split(":")[-1].strip()
        row["Provisional"] = methods[2].text.split(":")[-1].strip()
        # append that row to the overall data
        data.append(row)

# turn into df
df = pd.DataFrame(data)

# save as csv
df.to_csv('Data/data'+ts+'.csv')

In [110]:
df

,County,Candidate,Party,Share,Votes,Election Day,Mail,Provisional
0,ADAMS,"BIDEN, JOSEPH ROBINETTE JR",DEM,32.20%,"18,207",6611,11436,160
1,ADAMS,"TRUMP, DONALD J.",REP,66.37%,"37,523",29942,7148,433
2,ADAMS,"JORGENSEN, JO",LIB,1.43%,810,544,256,10
3,ALLEGHENY,"BIDEN, JOSEPH ROBINETTE JR",DEM,59.61%,"429,065",148171,273080,7814
4,ALLEGHENY,"TRUMP, DONALD J.",REP,39.23%,"282,324",209459,66575,6290
...,...,...,...,...,...,...,...,...
196,WYOMING,"TRUMP, DONALD J.",REP,66.87%,"9,936",8317,1503,116
197,WYOMING,"JORGENSEN, JO",LIB,1.47%,218,159,54,5
198,YORK,"BIDEN, JOSEPH ROBINETTE JR",DEM,36.95%,"88,114",35869,50122,2123
199,YORK,"TRUMP, DONALD J.",REP,61.53%,"146,733",119209,24554,2970


# Once Again, but for State Results as They'll Actually Appear

In [145]:
supreme = '/General/CountyBreakDownResults?officeId=8&districtId=1&ElectionID=undefined&ElectionType=undefined&IsActive=undefined'
superior = '/General/CountyBreakDownResults?officeId=9&districtId=1&ElectionID=undefined&ElectionType=undefined&IsActive=undefined'
comm = '/General/CountyBreakDownResults?officeId=10&districtId=1&ElectionID=undefined&ElectionType=undefined&IsActive=undefined'

pages = {
    'supreme':supreme,
    'superior':superior,
    'comm':comm
}

In [223]:
# get pages and save as 

browser = webdriver.Chrome()

docs = []

for key, value in pages.items():
    browser.get(base+value)
    time.sleep(3)
    html = browser.page_source
    # get timestamp
    ts = datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
    doc = BeautifulSoup(html)

    # write html to the file with the time
    # define path
    file_path = 'HTMLs/page_'+key+'-'+ts+'.html'
    html = doc.prettify()
    # open and save to PDFs folder with the filename we saved earlier
    with open(file_path, 'w') as f:
        f.write(html)
        
    # find just the table we want
    table = doc.find('div',class_='container').find('div',{'ng-controller':'CountyBreakDownGeneralController'})\
        .find('div',class_='panel-body').find('div',class_='col-xs-9').find('div',class_='panel-body')\
        .find_all('div',{'ng-repeat':'districts in filterRecords(levels)'})

    data_standard = []
    data_retention = []

    for elem in table:
        # get county value from header
        county = elem.find('h5').text.strip()
        
        if "Retention" in county:
            county = county.split(" ")[0]
            # each of the 'item in district' divs represents a candidate
            cands = elem.find_all('div',{'ng-repeat':'item in districts'})
            for cand in cands:
                # one row of the final df for each candidate for each county
                row = {}
                row["County"] = county
                # each candidate contains two 'rows' of data
                r = cand.find_all('div',class_='row')
                # the first 'row' [0] contains candidate name, party, vote share, and raw votes
                row["Candidate"] = r[0].find(class_='col-sm-4').find_all('span')[0].text.strip()
                # no party data worth sharing, so skip that, as compared to standard partisan elections
                # separate Yes and No shared
                shares = r[0].find(class_='col-sm-5').find_all('div',class_='progress-bar')
                row["Yes Share"] = shares[0].text.strip()
                row["No Share"] = shares[1].text.strip()
                # split up the Votes section by colon marks/ spaces and parse accordingly
                # note that we have to split remove a pesky unicode character
                row["Yes Votes"] = r[0].find(class_='pull-left').text.split(":")[2].split("\xa0")[1].strip()
                row["No Votes"] = r[0].find(class_='pull-left').text.split(":")[3].strip()
                # the second 'row' [1] contains vote breakdown by method
    #             methods = r[1].find_all('div',class_='col-sm-4')
    #             row["Election Day"] = methods[0].text.split(":")[-1].strip()
    #             row["Mail"] = methods[1].text.split(":")[-1].strip()
    #             row["Provisional"] = methods[2].text.split(":")[-1].strip()
                # append that row to the overall data
                data_retention.append(row)
        else:
            # each of the 'item in district' divs represents a candidate
            cands = elem.find_all('div',{'ng-repeat':'item in districts'})
            for cand in cands:
                # one row of the final df for each candidate for each county
                row = {}
                row["County"] = county
                # each candidate contains two 'rows' of data
                r = cand.find_all('div',class_='row')
                # the first 'row' [0] contains candidate name, party, vote share, and raw votes
                details = r[0].find(class_='col-sm-4').find_all('span')
                row["Candidate"] = details[0].text.strip()
                row["Party"] = details[1].text.strip().replace("(","").replace(")","")
                row["Share"] = r[0].find(class_='col-sm-5').text.strip()
                row["Votes"] = r[0].find(class_='col-sm-3').text.split(":")[-1].strip()
                # the second 'row' [1] contains vote breakdown by method
    #             methods = r[1].find_all('div',class_='col-sm-4')
    #             row["Election Day"] = methods[0].text.split(":")[-1].strip()
    #             row["Mail"] = methods[1].text.split(":")[-1].strip()
    #             row["Provisional"] = methods[2].text.split(":")[-1].strip()
                # append that row to the overall data
                data_standard.append(row)

    # turn into df
    df_s = pd.DataFrame(data_standard)
    df_r = pd.DataFrame(data_retention)

    # save as csvs, provided the df_r one actually exists 
    df_s.to_csv('Data/data_'+key+'_partisan_'+ts+'.csv',index=False)
    if len(df_r) > 0:
        df_r.to_csv('Data/data_'+key+'_retention_'+ts+'.csv',index=False)

# County Page Results

In [ ]:
# capture timestamp the data is pulled-- DONE
# write html to .txt doc-- DONE
# run the script
# save to csv, with title including the timestamp-- DONE

# run script every 15 min on Github Actions